In [1]:
import numpy as np
import deepchem as dc
import numpy as np
from deepchem.feat import GraphData
import pandas as pd
from deepchem.feat import DMPNNFeaturizer
import deepchem as dc


No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\deepchem\models\torch_models\__init__.py)


In [2]:
def get_graphs(weighted_adj_matrix):
    node_features = weighted_adj_matrix.diagonal().reshape(-1, 1)
    edges = np.array(np.nonzero(weighted_adj_matrix))
    edge_weights = weighted_adj_matrix[edges.T[:, 0], edges.T[:, 1]]
    graph = GraphData(node_features=node_features, edge_index=edges, edge_features=edge_weights.reshape(-1, 1))
    return graph

In [3]:
TASKS = [
'alcoholic', 'aldehydic', 'alliaceous', 'almond', 'amber', 'animal',
'anisic', 'apple', 'apricot', 'aromatic', 'balsamic', 'banana', 'beefy',
'bergamot', 'berry', 'bitter', 'black currant', 'brandy', 'burnt',
'buttery', 'cabbage', 'camphoreous', 'caramellic', 'cedar', 'celery',
'chamomile', 'cheesy', 'cherry', 'chocolate', 'cinnamon', 'citrus', 'clean',
'clove', 'cocoa', 'coconut', 'coffee', 'cognac', 'cooked', 'cooling',
'cortex', 'coumarinic', 'creamy', 'cucumber', 'dairy', 'dry', 'earthy',
'ethereal', 'fatty', 'fermented', 'fishy', 'floral', 'fresh', 'fruit skin',
'fruity', 'garlic', 'gassy', 'geranium', 'grape', 'grapefruit', 'grassy',
'green', 'hawthorn', 'hay', 'hazelnut', 'herbal', 'honey', 'hyacinth',
'jasmin', 'juicy', 'ketonic', 'lactonic', 'lavender', 'leafy', 'leathery',
'lemon', 'lily', 'malty', 'meaty', 'medicinal', 'melon', 'metallic',
'milky', 'mint', 'muguet', 'mushroom', 'musk', 'musty', 'natural', 'nutty',
'odorless', 'oily', 'onion', 'orange', 'orangeflower', 'orris', 'ozone',
'peach', 'pear', 'phenolic', 'pine', 'pineapple', 'plum', 'popcorn',
'potato', 'powdery', 'pungent', 'radish', 'raspberry', 'ripe', 'roasted',
'rose', 'rummy', 'sandalwood', 'savory', 'sharp', 'smoky', 'soapy',
'solvent', 'sour', 'spicy', 'strawberry', 'sulfurous', 'sweaty', 'sweet',
'tea', 'terpenic', 'tobacco', 'tomato', 'tropical', 'vanilla', 'vegetable',
'vetiver', 'violet', 'warm', 'waxy', 'weedy', 'winey', 'woody'
]
print("No of tasks: ", len(TASKS))

No of tasks:  138


In [4]:
import deepchem as dc
from openpom.feat.graph_featurizer import GraphFeaturizer, GraphConvConstants
from openpom.utils.data_utils import get_class_imbalance_ratio

In [6]:
from deepchem.feat import DMPNNFeaturizer
import deepchem as dc


featurizer = DMPNNFeaturizer()

smiles_field = 'nonStereoSMILES'


loader = dc.data.CSVLoader(
    tasks=TASKS,  
    feature_field=smiles_field,
    featurizer=featurizer
)



In [7]:

dataset = loader.create_dataset(inputs=["DMPNNpruned_without hydrogen_curated_GS_LF_merged_4812_QM_cleaned.csv"])
n_tasks = len(dataset.tasks)
dataset.X
len(dataset.X)


4812

In [8]:
data = np.load("DMPNNpruned_graph_data_cleaned.npz",allow_pickle=True)
mtx_list = data['mtx']
qm_graphs = []
for mtx in mtx_list:
    qm_graphs.append(get_graphs(mtx))
QM_X = np.asarray(qm_graphs)
QM_X
len(QM_X)


4812

In [9]:
X = np.asarray([i for i in zip(dataset.X, QM_X)])
new_dataset = dc.data.NumpyDataset(X=X, y=dataset.y, n_tasks=138, ids=dataset.ids)

In [10]:
new_dataset

<NumpyDataset X.shape: (4812, 2), y.shape: (4812, 138), w.shape: (4812, 1), task_names: [  0   1   2 ... 135 136 137]>

In [14]:
new_dataset

<NumpyDataset X.shape: (4812, 2), y.shape: (4812, 138), w.shape: (4812, 1), task_names: [  0   1   2 ... 135 136 137]>

In [ ]:

randomstratifiedsplitter = dc.splits.RandomStratifiedSplitter()
train_dataset, test_dataset, valid_dataset = randomstratifiedsplitter.train_valid_test_split(new_dataset, frac_train = 0.8, frac_valid = 0.1, frac_test = 0.1, seed = 1)

In [16]:
print(len(train_dataset))
print(len(valid_dataset))
print(len(test_dataset))

3839
485
488


In [17]:
dc.data.DiskDataset.from_numpy(X=train_dataset.X,
                    y= train_dataset.y,
                    w= train_dataset.w,
                    ids=train_dataset.ids,
                    tasks=TASKS,
                    data_dir="./train_split")

<DiskDataset X.shape: (3839, 2), y.shape: (3839, 138), w.shape: (3839, 1), task_names: ['alcoholic' 'aldehydic' 'alliaceous' ... 'weedy' 'winey' 'woody']>

In [18]:
dc.data.DiskDataset.from_numpy(X=valid_dataset.X,
                    y= valid_dataset.y,
                    w= valid_dataset.w,
                    ids=valid_dataset.ids,
                    tasks=TASKS,
                    data_dir="./valid_split")

<DiskDataset X.shape: (485, 2), y.shape: (485, 138), w.shape: (485, 1), ids: ['CCC(=O)C(=O)O' 'O=C(O)CCCCC(=O)O' 'O=C(O)c1ccccc1' ...
 'CC(C)C(NC(=O)CCC(N)C(=O)O)C(=O)NCC(=O)O' 'CCC1SC(CC(C)C)=NC1C'
 'O=c1[nH]cnc2c1ncn2C1OC(COP(=O)(O)O)C(O)C1O'], task_names: ['alcoholic' 'aldehydic' 'alliaceous' ... 'weedy' 'winey' 'woody']>

In [19]:
dc.data.DiskDataset.from_numpy(X=test_dataset.X,
                    y= test_dataset.y,
                    w= test_dataset.w,
                    ids=test_dataset.ids,
                    tasks=TASKS,
                    data_dir="./test_split")

<DiskDataset X.shape: (488, 2), y.shape: (488, 138), w.shape: (488, 1), ids: ['CC(O)CN' 'O=Cc1ccccc1' 'CC(=O)C(C)=O' ... 'CCCCCCCC(C)=NO'
 'CCCC=CC1OCC(O)CO1' 'CC(C)C1CCC2CC(=O)CC(C)C2(C)C1'], task_names: ['alcoholic' 'aldehydic' 'alliaceous' ... 'weedy' 'winey' 'woody']>

In [27]:
type(train_dataset)
import os
import deepchem as dc

def convert_to_disk_dataset(train_dataset, output_dir):
    """
    Converts a deepchem.data.datasets.NumpyDataset to a deepchem.data.datasets.DiskDataset.
    
    Parameters:
    train_dataset (deepchem.data.datasets.NumpyDataset): The input NumpyDataset to convert.
    output_dir (str): The directory to save the DiskDataset.
    
    Returns:
    deepchem.data.datasets.DiskDataset: The converted DiskDataset.
    """
    os.makedirs(output_dir, exist_ok=True)
    
    X, y, w, ids = train_dataset.X, train_dataset.y, train_dataset.w, train_dataset.ids
    
    disk_dataset = dc.data.DiskDataset.from_numpy(X, y, w, ids, data_dir=output_dir)
    
    return disk_dataset
output_dir = r'C:\Users\sarab\OneDrive\Desktop\hyperparamter_dmpnn\train_dataset'
train_dataset = convert_to_disk_dataset(train_dataset, output_dir)

In [28]:
train_ratios = get_class_imbalance_ratio(train_dataset)
assert len(train_ratios) == n_tasks

In [ ]:
from deepchem.models.optimizers import ExponentialDecay

learning_rate = ExponentialDecay(initial_rate=0.005390333, decay_rate=0.777099289, decay_steps=764, staircase=True)


In [30]:
model = MPNNPOMModel(n_tasks = n_tasks,
                            batch_size=25,
                            learning_rate=learning_rate,
                            class_imbalance_ratio = train_ratios,
                            loss_aggr_type = 'sum',
                            node_out_feats = 25,
                            edge_hidden_feats = 75,
                            edge_out_feats = 25,
                            num_step_message_passing = 5,
                            mpnn_residual = True,
                            message_aggregator_type = 'max',
                            mode = 'classification',
                            number_atom_features = 133,
                            number_bond_features = 14,
                            n_classes = 1,
                            readout_type = 'set2set',
                            num_step_set2set = 3,
                            num_layer_set2set = 2,
                            ffn_hidden_list= [512, 512],
                            ffn_embeddings = 256,
                            ffn_activation = 'relu',
                            ffn_dropout_p = 0.154493949,
                            ffn_dropout_at_input_no_act = False,
                            weight_decay = 1.39e-6,
                            self_loop = False,
                            optimizer_name = 'adam',
                            log_frequency = 32,
                            model_dir = './examples/experiments',
                            device_name='cpu')

In [31]:
nb_epoch = 60

In [32]:
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)

In [ ]:

for epoch in range(1, nb_epoch+1):
    loss = model.fit(
            train_dataset,
            nb_epoch=1,
            max_checkpoints_to_keep=1,
            deterministic=False,
            restore=epoch>1)
    train_scores = model.evaluate(train_dataset, [metric])['roc_auc_score']
    valid_scores = model.evaluate(valid_dataset, [metric])['roc_auc_score']
    print(f"epoch {epoch}/{nb_epoch} ; loss = {loss}; train_scores = {train_scores}; valid_scores = {valid_scores}")


c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\deepchem\feat\graph_data.py:193: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:212.)
  g.ndata['x'] = torch.from_numpy(self.node_features).float()


epoch 1/60 ; loss = 2.928974151611328; train_scores = 0.7419070435573598; valid_scores = 0.725398866190326


c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\deepchem\models\torch_models\torch_model.py:1078: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(check

epoch 2/60 ; loss = 2.705685043334961; train_scores = 0.7693421291111174; valid_scores = 0.7452113275225853


c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\deepchem\models\torch_models\torch_model.py:1078: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(check

...

epoch 58/60 ; loss = 1.975441813468933; train_scores = 0.9313254783091818; valid_scores = 0.8687646881667459


c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\deepchem\models\torch_models\torch_model.py:1078: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(check

epoch 59/60 ; loss = 2.17451171875; train_scores = 0.932499865084038; valid_scores = 0.8706614944898889


c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\deepchem\models\torch_models\torch_model.py:1078: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(check

epoch 60/60 ; loss = 2.1538979212443032; train_scores = 0.9322938100083407; valid_scores = 0.8695557095350969


In [ ]:
test_scores = model.evaluate(test_dataset, [metric])['roc_auc_score']

print("test_score: ", test_scores)

test_score:  0.8654902641917108
